In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [ ]:
import torch
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Tile_Dataset import Vesuvius_Tile_Datamodule
import matplotlib.patches as patches
from lit_models.UNET_TILE import UNET_TILE_lit
from monai.visualize import matshow3d
import einops
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")


In [ ]:
PATCH_SIZE = 224
Z_DIM = 8

class CFG:
    
    train_fragment_id=[2,3]
    val_fragment_id=[1]
    batch_size = 32
    patch_size = PATCH_SIZE
    z_dim = Z_DIM
    stride = patch_size // 2
    #comp_dataset_path = COMPETITION_DATA_DIR
    num_workers = 0
    on_gpu = True

In [ ]:
dataset = Vesuvius_Tile_Datamodule(cfg=CFG)

dataloader = iter(dataset.train_dataloader())
for i in range(1):
    # Get image and label from train data -- change number for different ones
    subvolumes, inklabels = next(dataloader)
    print('subvolume shape:',subvolumes.shape)
    print('inklabel shape:',inklabels.shape)
    for subvolume,  inklabel in zip(subvolumes, inklabels):
                fig, axes = plt.subplots(1, 2, figsize=(15, 5))
                for idx, image in enumerate((subvolume,  inklabel)):
                    if idx==0:
                        axes[idx].imshow(image[0])
                    else:
                         axes[idx].imshow(image.squeeze(0))

                plt.show()
    

plot_dataset = dataset.train_dataloader()
plot_count = 0
for i in range(1000):
    image, mask = plot_dataset[i]
    #data = transform(image=image, mask=mask)
    aug_image = image#.squeeze(0) # data['image']
    aug_mask = mask#.squeeze(0) #data['mask']
    print(image.shape, mask.shape)

    if mask.sum() == 0:
        continue

    fig, axes = plt.subplots(1, 4, figsize=(15, 8))
    axes[0].imshow(image[..., 0], cmap="gray")
    axes[1].imshow(mask, cmap="gray")
    #axes[2].imshow(aug_image[..., 0], cmap="gray")
    #axes[3].imshow(aug_mask, cmap="gray")
    
    #plt.savefig(CFG.figures_dir + f'aug_fold_{CFG.valid_id}_{plot_count}.png')

    plot_count += 1
    if plot_count == 5:
        break
    
    

In [ ]:
lit_model = UNET_TILE_lit(
        use_wandb = True,
        z_dim = Z_DIM,
        patch_size = (PATCH_SIZE,PATCH_SIZE),
        sw_batch_size=8 ,
        eta_min = 1e-8,
        t_max = 120,
        max_epochs = 300,
        weight_decay =  0.0001,
        learning_rate = 0.00001,
        gamma = 0.85,)


FROM_CHECKPOINT = F
if FROM_CHECKPOINT:
    lit_model = lit_model.load_from_checkpoint('logs/Local_Originalgood_1/last-v1.ckpt', 
                                              #patch_size = (512,512),
                                              #learning_rate = 0.0001,
                                               #weight_decay =  .001,
                                               #eta_min = 1e-7,
                                               #max_epochs = 100,
                                             #sw_batch_size = 8
                                              )

In [ ]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="fbeta_4",
    mode="max",
    dirpath="logs/BCE2+05Tversky_Local_Originalgood",
    filename="BCE2+05Tversky_Local_Originalgood{epoch:02d}{val_loss:.2f}{fbeta_4:.2f}{recall:.2f}{precision:.2f}",
    save_last =True,
)


trainer = pl.Trainer(
        accelerator='mps',
        #benchmark=True,
        max_epochs=120,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/BCE2+05Tversky_Local_Originalgood/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/BCE2+05Tversky_Local_Originalgood/',
        #overfit_batches=1,
        #precision=16,
        accumulate_grad_batches=1, 
        callbacks=[checkpoint_callback],
        #resume_from_checkpoint ='logs/smp_unet_32_.5/lightning_logs/version_3/checkpoints/FocalDice_768epoch=123FBETA=0.30recall=0.77precision=0.26.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/Local_SMPEffB3_Tile_12_224patch/last.ckpt'
           )
# resume_from_checkpoint = 
#ckpt_path='logs/unet_smp-epoch=102-val_loss=0.00.ckpt'
# ckpt_path='logs/Eff_monai_32z/lightning_logs/version_0/checkpoints/epoch=19-step=40.ckp'